<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2023<br>
<font color=3C99D size=5>
    Practical Assignment 2 - CSP <br>
<font color=696880 size=4>
    Yalda Shabanzadeh, Ali Salesi

____

# Personal Data

In [1]:
# Set your student number
student_number = '400104964'
name = 'Aylin'
last_name = 'Rasteh'

# Rules
- Make sure that all of your cells can be run perfectly. 

# Q1: Cryptarithmetic Puzzle (100 Points)

<font size=4>
Author: Yalda Shabanzadeh, Ali Salesi
<br/>
<font color=red>
Please run all the cells.
</font>
</font>
<br/>
</div>

## Problem Definition

The Cryptarithmetic Puzzle is a puzzle where the digits of some numbers are replaced with letters to make a mathematical equation. Each letter represents a unique digit from 0 to 9 (in case of base 10). The puzzle has the following constraints:

- Each letter can be replaced by a digit from 0 to 9 (in case of base 10), but no digit can be used twice.
- Each letter must be replaced by the same digit throughout the puzzle.
- The leftmost digit of each number cannot be zero.

### Example
Here's an example puzzle:

```
  two
+ two
------
 four
```

In this puzzle, we need to replace each letter with a digit from 0 to 9 (in case of base 10) to make the equation true.

## Note
- You can use any arbitrary CSP algorithm to solve this question.
- Allowed algorithms:
BackTracking, AC3, AC3-LCV, AC3-MRV, AC3-LCV-MRV

> Explain how you consider this problem as a CSP (5 points)


<font color='cyan'>
In a cryptarithmetic problem each letter stands for a distinct digit. The Goal is to find a substitution of digits for letters such that the resulting sum is arithmetically correct, with the added restriction that no leading zeroes are allowed.
</font>
<br>
<font color='cyan'>
Variables: All given alphabets
</font>
<br>
<font color='cyan'>
Domain: Integers from 0 to N-1
</font>
<br>
<font color='cyan'>
Constraints: Alldiff constraint and column addition constraints and nonzero constraint
</font>

## Input
  - The first line is $N$ ,base of the sum operation. $2 \le N \le 36$. implementing CSP for $N$ other than 10 is optional and test cases with $N$ more than 10 has bonus score since it has a bigger search dimension so AC3, MRV and LCV is suggested.
  - In the following lines until the last, operands for the sum operation is given.
  - In the last line, $result$, the sum of strings is given.
  - **Note**: all the operands and the result are in lowercase.
 
$$1 \le N \le 50$$

### Sample Input
This sample describes the previous example.
```
10
two
two
four
```

## Output
for each given charachter as input, print it's corresponding number:
  - **Note**: order is not important. there are multiple solutions for this problem. if all the constraints are satisfied, you can return any solution.

### Sample Output
The solution to the example above is:
```
f 1
o 4
r 8
t 7
u 6
w 3
```

## Your code
**Note:** It's OK to change the signature for the given functions and the given structure is just a suggestion to help you with the implementation. If you tried to use another structure, the total score of that part won't be changed.

### Define Constraint (20 points)

In this part, write the satisfaction part of the constraint of this problem.
You can add any argument you want to `__init__` function.


- Check if all letters have a different value. (Unique Constraint)
- Check if the first letter of each word is not 0. (Not Zero Constraint)
- Check if the sum of the operands with assigned values is equal to the result. (Sum Constraint)
    - **Note**: if there are unassigned values for variables, you can ignore them and return true. for large inputs, you can use AC3 or MRV, LCV to reduce the search space with partial assignment.

In [2]:
import numpy as np


class Constraint:
    """
    Base class for all constraints.
    """

    def __init__(self, variable: str) -> None:
        """
        Initializes the constraint.

        Args:
            variable: name of the variable
        """
        self.variable = variable

    def satisfied(self, assignment: dict[str, int]) -> bool:
        """
        Checks if the constraint is satisfied.

        Args:
            assignment: dictionary of the assignment of the variables to the values

        Returns:
            False if the constraint is not satisfied, True otherwise (including if the variable is not assigned)
        """
        return True


class NotZeroConstraint(Constraint):
    """"
    Constraint that checks if the variable is not zero.
    """

    def satisfied(self, assignment: dict[str, int]) -> bool:
        if self.variable not in assignment:
            return True
        return assignment[self.variable] != 0


class UniqueConstraint(Constraint):
    """
    Constraint that checks if all the variables has unique values.
    """

    def __init__(self) -> None:
        pass

    def satisfied(self, assignment: dict[str, int]) -> bool:
        return len(assignment.keys()) == len(set(assignment.values()))


class SumConstraint(Constraint):
    """
    Constraint that checks if the sum of the operands is equal to the result.
    """

    def __init__(self, operands: list[str], result: str, base: int = 10, allow_carry: bool = False) -> None:
        """
        initializes the constraint.

        Args:
            operands: list of the operands
            result: result of the sum
            base: base of the numbers
            allow_carry: if true, the variables in the result is checked from right to left so sum of the operands can have a carry,
                         it is useful for partial sums of columns in AC3 or backtracking. implementation of this part is not required.
        """
        self.operands = operands
        self.result = result
        self.base = base
        self.allow_carry = allow_carry
        

    def satisfied(self, assignment: dict[str, int]) -> bool:
        #TODO: implement this method
        continuee = True
        base = self.base
        operands = self.operands
        result = self.result
        c = 0
        i = 1
        while continuee:
            continuee = False
            summ = 0
            for operand in operands:
                lenn = len(operand)
                if lenn >= i:
                    continuee = True
                    if operand[lenn - i] not in assignment:
                        return True
                    summ += assignment[operand[lenn - i]]
            summ += c
            s = summ % base
            c = summ // base
#             print(1)
            if len(result) >= i:
                if result[len(result) - i] in assignment:
#                     print("sum:", summ, " i:", i, " s:", s, " c:", c, " ass:", assignment[result[len(result) - i]])
                    if assignment[result[len(result) - i]] != s:
                        return False
            if not continuee:
                if len(result) - i == 0 and c != 0:
                    return False
                while len(result) - i > 0:
                    i += 1
                    s = c % base
                    c = c // base
                    if result[len(result) - i] in assignment:
                        if assignment[result[len(result) - i]] != s:
                            return False
                    if len(result) - i == 0 and c != 0:
                        return False    
            i += 1 
#         i = 1
#         summ = 0
#         for operand in operands:
#             lenn = len(operand)
#             if lenn >= i:
#                 continuee = True
#                 if operand[lenn - i] not in assignment:
#                     return True
#                 summ += assignment[operand[lenn - i]]
#             print("sum: ", summ, operand)

#         sumVariables = 0
#         for operand in self.operands:
#             n = 0
#             for o in operand:
#                 n *= self.base
#                 if o not in assignment:
#                     return True
#                 n += assignment[o]
#             numbers.append(n)
#             summ += n
#         res = 0
#         for o in self.result:
#             res *= self.base
#             res += assignment[o]
                
        return True

### Define a CSP class (50 points)

You can add functions you'll need to this class.

In [3]:
class CSP:
    """
    CSP class that contains the variables, constraints and base of the problem.
    """

    def __init__(self, variables: list[str], constraints: list[Constraint], base: int = 10) -> None:
        self.variables = variables
        self.constraints = constraints
        self.base = base

    def consistent(self, assignment: dict[str, int]) -> bool:
        """
        Checks if the assignment is consistent with the constraints.

        Args:
            assignment: dictionary of the assignment of the variables to the values.

        Returns:
            False if the assignment is not consistent, True otherwise
        """
        for constraint in self.constraints:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment, domains = None, i = 0):
        """
        Backtracking search algorithm.

        Args:
            assignment: dictionary of the assignment of the variables to the values. assignment is modified in place.

        Returns:
            True if a solution is found, False otherwise
        """
        #TODO: implement this method
        variables = self.variables
        if domains == None:
            domains = {}
            for v in variables:
                domains[v] = list(range(0, self.base))
                
#         if not self.consistent(assignment):
#         print("assignment", assignment)
        if not self.consistent(assignment):
            return False
        if (i >= len(variables)):
#             print("yes", assignment)
            return True
        variable = variables[i]
        ok = False
        for d in domains[variable]:
#             print(d, variable)
            assignment[variable] = d
            domainsCopy = domains.copy()
            if self.backtracking_search(assignment, domainsCopy, i+1):
                ok = True
                break
        if not ok:
            del(assignment[variable])
        return ok
    
    
    def forwardCheck(self, assignment, domains):
        variables = self.variables
        constraints = self.constraints
        assignmentCopy = assignment.copy()   
#         domainsCopy = domains.copy()
        for v in variables:
            if v not in assignment:
                domainCopy = domains[v].copy()
                for d in domains[v]:
                    assignmentCopy[v] = d
#                     print(d, v, domains[v])
                    if not self.consistent(assignmentCopy):
                        domainCopy.remove(d)
#                         print("deleted: ", d, v, assignmentCopy)
                if len(domainCopy) == 0:
                    return False
                del(assignmentCopy[v])
                domains[v] = domainCopy
#         print("domains: ", domains)
        return True
                            

### Build CSP and solve it! (20 points)

complete these functions for testing.

In [4]:
import copy
def get_problem(operands: list[str], result: str, base: int = 10) -> CSP:
    """
    Creates a CSP problem from the operands and the result. The variables are the unique characters in the operands and the result.
    The constraints should be defined usinng NotZeroConstraint for leftmost digits, a UniqueConstraint for all variables and a SumConstraint for the problem
    multiple partial SumConstraint can be used for bigger search spaces and base > 10.

    Args:
        operands: list of the operands
        result: result of the sum
        base: base of the numbers

    Returns:
        CSP problem
    """
      
    #TODO: implement this method
    variables = []
#     for operand in operands:
#         for o in operand:
#             if o not in variables:
#                 variables.append(o)
#     for o in result:
#         if o not in variables:
#             variables.append(o)  
    continuee = True
    operandsCopy = copy.deepcopy(operands)
    operandsCopy.append(result)
    i = 1
    while continuee:
        continuee = False
        for operand in operandsCopy:
            lenn = len(operand)
            if lenn >= i:
                continuee = True
                if operand[lenn - i] not in variables:
                    variables.append(operand[lenn - i])  
        i += 1 
            
#     variables = sorted(variables)
    constraints = []
    for operand in operands:
        constraints.append(NotZeroConstraint(operand[0]))
    constraints.append(NotZeroConstraint(result[0]))
    constraints.append(UniqueConstraint())
    constraints.append(SumConstraint(operands, result, base, False))
    return CSP(variables, constraints, base)


In [5]:
def backtracking_search(csp: CSP) -> dict[str, int]:
    """
    Backtracking search algorithm.

    Args:
        csp: CSP problem
    
    Returns:
        dictionary of the assignment of the variables to the values if a solution is found, None otherwise
    """
    sol = {}
    ok = csp.backtracking_search(sol)
    if ok:
        myKeys = list(sol.keys())
        myKeys.sort()
#         print(myKeys)
        sorted_dict = {i: sol[i] for i in myKeys}
        print(sorted_dict)
        return sorted_dict
    return None

base = 10
operands = ["point", "zero"]
result = "energy"
csp = get_problem(operands, result, base)
sol = backtracking_search(csp)

{'e': 1, 'g': 7, 'i': 5, 'n': 0, 'o': 8, 'p': 9, 'r': 6, 't': 4, 'y': 2, 'z': 3}


## Test

**Do not change the following cell.**

**Note**: 5 tests will be added after deadline. (5 points).

In [6]:
import helper.test_q2 as q2
import time

TIME_LIMIT = 3

tests = q2.get_all_tests(prefix='q2_')
tests_passed = 0
for test in tests:
    base, operands, result = q2.scan_test_input(test)
    csp = get_problem(operands, result, base)
    start_time = time.time()
    sol = backtracking_search(csp)
    if sol is None:
        print(f'test {test} failed. time elapsed={time.time() - start_time:.5f}s')
        continue
    total_time = time.time() - start_time
    if q2.is_result_valid(sol, operands, result, base) and total_time < TIME_LIMIT:
        tests_passed += 1
        print(f'test {test} passed. time elapsed={total_time:.5f}s')
        print('solution: ')
        print('\n'.join([str(x).upper() + " " + str(y)
              for x, y in sorted(sol.items())]))
    else:
        print(f'test {test} failed. time elapsed={total_time:.5f}s')
print(f'Score = {tests_passed / len(tests) * 100}%')

{'a': 2, 'f': 21, 'i': 29, 'l': 3, 'o': 1, 'u': 25, 'w': 4, 'y': 0}
test q2_in8.txt passed. time elapsed=0.01424s
solution: 
A 2
F 21
I 29
L 3
O 1
U 25
W 4
Y 0
{'a': 2, 'f': 8, 'l': 4, 'w': 5, 'y': 0}
test q2_in4.txt passed. time elapsed=0.00048s
solution: 
A 2
F 8
L 4
W 5
Y 0
{'g': 8, 'o': 1, 't': 2, 'u': 0}
test q2_in5.txt passed. time elapsed=0.00007s
solution: 
G 8
O 1
T 2
U 0
{'e': 1, 'g': 7, 'i': 5, 'n': 0, 'o': 8, 'p': 9, 'r': 6, 't': 4, 'y': 2, 'z': 3}
test q2_in7.txt passed. time elapsed=0.11655s
solution: 
E 1
G 7
I 5
N 0
O 8
P 9
R 6
T 4
Y 2
Z 3
{'d': 7, 'e': 5, 'm': 1, 'n': 6, 'o': 0, 'r': 8, 's': 9, 'y': 2}
test q2_in6.txt passed. time elapsed=0.03211s
solution: 
D 7
E 5
M 1
N 6
O 0
R 8
S 9
Y 2
{'a': 1, 'b': 8, 'c': 9, 'd': 2, 'e': 0}
test q2_in2.txt passed. time elapsed=0.00164s
solution: 
A 1
B 8
C 9
D 2
E 0
{'a': 1, 'b': 9, 'c': 8}
test q2_in3.txt passed. time elapsed=0.00202s
solution: 
A 1
B 9
C 8
{'x': 1, 'y': 2, 'z': 8}
test q2_in1.txt passed. time elapsed=0.00006s
s